In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip uninstall genecnv

In [3]:
!pip install git+https://github.com/nik548/cscbfinalprojectS25.git

  Cloning https://github.com/nik548/cscbfinalprojectS25.git to /tmp/pip-req-build-vgqc0qw_
  Running command git clone --filter=blob:none --quiet https://github.com/nik548/cscbfinalprojectS25.git /tmp/pip-req-build-vgqc0qw_
  Resolved https://github.com/nik548/cscbfinalprojectS25.git to commit e9c70dd6b5b580a492ab6f80932a38f375c342ff
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 144.5/144.5 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 165.9/165.9 kB 12.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 48.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 55.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.3/13.3 MB 71.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.1/53.1 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.7/46.7 kB 3.1 MB/s eta 0:00:00
  Created wheel for genecnv: filename=genecnv-0.1.0-py3-none-any.whl size

In [4]:
import genecnv as genecnv

In [5]:
from genecnv import run_adaptive_cnv_pipeline, annotate_cnv_calls, annotate_genes_mygene, preprocess

In [6]:
!pip install scanpy python-igraph scipy anndata

In [7]:
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)
import os, sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import scanpy as sc
import anndata as ad
from sklearn.metrics import f1_score
import seaborn as sns
from typing import List, Dict, Optional, Tuple
from scipy.stats import zscore
from scipy.sparse import csr_matrix, issparse

In [8]:
data = ad.read_h5ad('/content/drive/MyDrive/CSCB_Final/Data/PBMC_simulated_cnas_041025.h5ad')

In [9]:
# (Optional) Annotate genes if coordinates missing
adata = annotate_genes_mygene(data)

# (Optional) Clean data
adata_clean = preprocess(adata,  allowed_chromosomes=['6', '22'] + ['X'])

INFO:biothings.client:querying 1-411 ...


Querying MyGene.info for 411 genes...


INFO:biothings.client:Finished.


After query, 358 genes remain un-annotated.
Dropped 358 genes without coordinates.
[QC] Filtered to 10259 cells
[QC] Retained 19425 genes after min_cells_per_gene=3
[QC] Retained 1991 genes with genomic coords
[Norm] Scaled to 10000.0 counts per cell
[Norm] Applied log1p
[Preprocess] Completed: 10259 cells × 1991 genes


In [11]:
import re

def compute_metrics(
    adata: sc.AnnData,
    gt_col: str,
    pred_col: str,
    overlap_fraction: float = 0.5
) -> pd.DataFrame:
    """
    Compute cell‑level confusion metrics by matching predicted vs. ground‑truth CNV regions,
    and return them as a 1‑row pandas DataFrame.

    Parameters
    ----------
    adata
        AnnData with CNV region strings in obs[gt_col] and obs[pred_col].
    gt_col, pred_col : str
        Column names in adata.obs for ground‑truth and predicted region annotations.
    overlap_fraction : float
        Minimum fraction of a ground‑truth segment that must be overlapped by a prediction
        (on the same chromosome) to count as a true positive.

    Returns
    -------
    pd.DataFrame
        One‑row table with columns [accuracy, precision, recall, specificity, TP, TN, FP, FN].
    """
    pattern = re.compile(r"([0-9XY]+):([0-9,]+)-([0-9,]+)")
    def parse(txt):
        out = []
        for m in pattern.finditer(txt or ""):
            chrom, s, e = m.groups()
            out.append((chrom, int(s.replace(",","")), int(e.replace(",",""))))
        return out

    TP = TN = FP = FN = 0
    for cell in adata.obs_names:
        gt = parse(adata.obs.at[cell, gt_col] or "")
        pr = parse(adata.obs.at[cell, pred_col] or "")
        pred_pos = len(pr) > 0
        truth_pos = True

        # require every GT interval to find a matching PR interval
        for cg, sg, eg in gt:
            matched = False
            length = eg - sg
            if length <= 0:
                truth_pos = False
                break
            for cp, sp, ep in pr:
                if cp != cg:
                    continue
                overlap = max(0, min(ep, eg) - max(sp, sg))
                if overlap / length >= overlap_fraction:
                    matched = True
                    break
            if not matched:
                truth_pos = False
                break

        if truth_pos and pred_pos:
            TP += 1
        elif not truth_pos and not pred_pos:
            TN += 1
        elif not truth_pos and pred_pos:
            FP += 1
        elif truth_pos and not pred_pos:
            FN += 1

    total = TP + TN + FP + FN
    acc = (TP + TN) / total if total else np.nan
    prec = TP / (TP + FP) if (TP + FP) else 0.0
    rec = TP / (TP + FN) if (TP + FN) else 0.0
    spec = TN / (TN + FP) if (TN + FP) else 0.0

    # return as a table
    return pd.DataFrame([{
        'accuracy':    acc,
        'precision':   prec,
        'recall':      rec,
        'specificity': spec,
        'TP':          TP,
        'TN':          TN,
        'FP':          FP,
        'FN':          FN
    }])



In [12]:
def tune_hyperparameters(
    adata: sc.AnnData,
    gt_col: str = 'simulated_cnvs',
    cell_type_key: str = 'cell_type',
    gene_bins: List[int] = [50,100,200],
    decay_scales: List[float] = [1e5,1e6,1e7],
    decay_radii: List[Optional[int]] = [5,10,20],
    ref_fracs: List[float] = [0.15],
    min_runs: List[int] = [3],
    sample_n: int = 500
) -> pd.DataFrame:
    """
    Tune hyperparameters on a fixed subset of sample_n cells that includes both CNV-positive
    and CNV-negative. Returns a DataFrame with one row per parameter combination and columns
    for accuracy, precision, recall, specificity.
    """
    # 1) select a reproducible subset containing both positives and negatives
    np.random.seed(0)
    has = adata.obs[gt_col].astype(bool)
    pos = adata.obs_names[has]
    neg = adata.obs_names[~has]
    n_pos = min(len(pos), max(1, sample_n // 4))
    n_neg = sample_n - n_pos
    subs_pos = np.random.choice(pos, n_pos, replace=False) if len(pos)>n_pos else pos
    subs_neg = np.random.choice(neg, n_neg, replace=False) if len(neg)>n_neg else neg
    subset = np.concatenate([subs_pos, subs_neg])
    ad_sub = adata[subset].copy()

    # 2) sweep and record metrics
    records = []
    for binsize in gene_bins:
        for decay_scale in decay_scales:
            for decay_radius in decay_radii:
                for ref_frac in ref_fracs:
                    for min_run in min_runs:
                        # run pipeline on the subset
                        ad_sub2, bins, centers, calls = run_adaptive_cnv_pipeline(
                            ad_sub,
                            cell_type_key=cell_type_key,
                            target_genes_per_bin=binsize,
                            decay_scale=decay_scale,
                            decay_radius=decay_radius,
                            reference_frac=ref_frac,
                            min_run=min_run
                        )
                        annotate_cnv_calls(ad_sub2, calls, bins, centers)
                        # compute_metrics returns a 1‑row DataFrame → extract its row
                        m_df = compute_metrics(ad_sub2, gt_col, 'cnv_regions')
                        m = m_df.iloc[0]

                        combo = (
                            f"bins={binsize},scale={decay_scale},"
                            f"rad={decay_radius},ref={ref_frac},run={min_run}"
                        )
                        print(
                            f"{combo} -> "
                            f"accuracy={m['accuracy']:.3f}, "
                            f"precision={m['precision']:.3f}, "
                            f"recall={m['recall']:.3f}, "
                            f"specificity={m['specificity']:.3f}"
                        )
                        records.append({
                            'params': combo,
                            'accuracy':    m['accuracy'],
                            'precision':   m['precision'],
                            'recall':      m['recall'],
                            'specificity': m['specificity']
                        })

    df = pd.DataFrame(records).set_index('params')
    return df


In [13]:
df = tune_hyperparameters(adata_clean, sample_n = 500)

bins=50,scale=100000.0,rad=5,ref=0.15,run=3 -> accuracy=0.786, precision=0.891, recall=0.860, specificity=0.270
bins=50,scale=100000.0,rad=10,ref=0.15,run=3 -> accuracy=0.826, precision=0.938, recall=0.860, specificity=0.561
bins=50,scale=100000.0,rad=20,ref=0.15,run=3 -> accuracy=0.650, precision=0.989, recall=0.606, specificity=0.953
bins=50,scale=1000000.0,rad=5,ref=0.15,run=3 -> accuracy=0.658, precision=0.940, recall=0.647, specificity=0.727
bins=50,scale=1000000.0,rad=10,ref=0.15,run=3 -> accuracy=0.748, precision=0.908, recall=0.798, specificity=0.345
bins=50,scale=1000000.0,rad=20,ref=0.15,run=3 -> accuracy=0.888, precision=0.928, recall=0.952, specificity=0.171
bins=50,scale=10000000.0,rad=5,ref=0.15,run=3 -> accuracy=0.250, precision=0.000, recall=0.000, specificity=1.000
bins=50,scale=10000000.0,rad=10,ref=0.15,run=3 -> accuracy=0.622, precision=0.962, recall=0.587, specificity=0.851
bins=50,scale=10000000.0,rad=20,ref=0.15,run=3 -> accuracy=0.816, precision=0.907, recall=0.

In [14]:
# ──────────────────────────────────────────────────────────────────────────────
# 11) Select best-balanced hyperparameters
# ──────────────────────────────────────────────────────────────────────────────

def pick_best_by_average(
    df: pd.DataFrame,
    metrics: List[str] = ['accuracy','precision','recall','specificity']
) -> Dict[str, float]:
    """
    Compute the mean of the specified metrics for each parameter combo,
    and return the combo with the highest average score.
    """
    df = df.copy()
    df['avg_score'] = df[metrics].mean(axis=1)
    best_idx = df['avg_score'].idxmax()
    res = df.loc[best_idx, metrics].to_dict()
    # parse params string into dict
    params = dict(item.split('=') for item in best_idx.split(','))
    # convert numeric types
    for k,v in params.items():
        try: params[k] = float(v)
        except: pass
    return {**params, **res}


def pick_best_by_minimum(
    df: pd.DataFrame,
    metrics: List[str] = ['accuracy','precision','recall','specificity']
) -> Dict[str, float]:
    """
    For each parameter combo, compute the minimum of the specified metrics,
    and return the combo that maximizes this minimum (maximin criterion).
    """
    df = df.copy()
    df['min_score'] = df[metrics].min(axis=1)
    best_idx = df['min_score'].idxmax()
    res = df.loc[best_idx, metrics].to_dict()
    params = dict(item.split('=') for item in best_idx.split(','))
    for k,v in params.items():
        try: params[k] = float(v)
        except: pass
    return {**params, **res}

In [15]:
display(df)
best_avg = pick_best_by_average(df)
print(best_avg)
best_min = pick_best_by_minimum(df)
print(best_min)

,accuracy,precision,recall,specificity
params,,,,
"bins=50,scale=100000.0,rad=5,ref=0.15,run=3",0.786,0.890995,0.860412,0.269841
"bins=50,scale=100000.0,rad=10,ref=0.15,run=3",0.826,0.938424,0.860045,0.561404
"bins=50,scale=100000.0,rad=20,ref=0.15,run=3",0.650,0.988764,0.605505,0.953125
"bins=50,scale=1000000.0,rad=5,ref=0.15,run=3",0.658,0.939799,0.647465,0.727273
"bins=50,scale=1000000.0,rad=10,ref=0.15,run=3",0.748,0.907928,0.797753,0.345455
"bins=50,scale=1000000.0,rad=20,ref=0.15,run=3",0.888,0.927813,0.952070,0.170732
"bins=50,scale=10000000.0,rad=5,ref=0.15,run=3",0.250,0.000000,0.000000,1.000000
"bins=50,scale=10000000.0,rad=10,ref=0.15,run=3",0.622,0.962121,0.586605,0.850746
"bins=50,scale=10000000.0,rad=20,ref=0.15,run=3",0.816,0.907029,0.886918,0.163265


{'bins': 100.0, 'scale': 100000.0, 'rad': 10.0, 'ref': 0.15, 'run': 3.0, 'accuracy': 0.818, 'precision': 1.0, 'recall': 0.803030303030303, 'specificity': 1.0}
{'bins': 100.0, 'scale': 1000000.0, 'rad': 10.0, 'ref': 0.15, 'run': 3.0, 'accuracy': 0.842, 'precision': 0.9870801033591732, 'recall': 0.8377192982456141, 'specificity': 0.8863636363636364}


From these results, we select the optimal combination of hyperparameters as:



*   **bins**: 100
*   **scale**: 1e6
*   **rad**: 10.0
*   **ref**: 0.15
*   **min_runs**: 3

These hyperparameter settings achieve the best trade‑off across all four evaluation metrics, so we adopted them for all subsequent analyses. Because the selected values fall near the center of each range we tested, we are confident that our search was sufficiently comprehensive.
